
   #### Processing Health Level 7 (HL7) Fast Healthcare Interoperability Resources (FHIR) semi-structured JSON Messages with Snowflake Snowpark, Cortex Analyst and Streamlit

   __FHIR__ was created by the standards development organization Health Level 7 (HL7) and was designed to enable health data, including clinical and administrative data, to be quickly and efficiently exchanged. 

   __FHIR R4__ is the most recent stable version of the FHIR standard, published in October 2019. It's a normative standard, meaning it's designed for forward compatibility and long-term use. FHIR R4 is widely adopted and powers many FHIR implementations today. 

   At the core of HL7 FHIR is a set of modular components called __Resources__. These form the basic data exchange format and model of FHIR. 
   Resources define the component data elements, constraints, and relationships that make up an exchangeable patient record.

Download and unzip __synthea_sample_data_fhir_r4_nov2021.zip__ file from FHIR R4 synthetic data from https://synthetichealth.github.io/synthea-sample-data/ website. The compressed archive contains synthetic HL7 FHIR R4 messages in JSON data files. 

Create the `database` and `schema` used for loading semi-structured data, and `virtual warehouse` as compute resource to load files into Snowflake and perform analytical queries.

In a separate Snowflake SQL worksheets run the SQL statements in the cell below.

In [ ]:
USE ROLE SYSADMIN;

CREATE OR REPLACE DATABASE HL7_FHIR 
    COMMENT = 'HL7 FHIR DATABASE';
    
CREATE OR REPLACE SCHEMA HL7_FHIR.HL7_FHIR_R4;

CREATE OR REPLACE WAREHOUSE HL7_FHIR_WH
    WAREHOUSE_SIZE = 'XSMALL'
    INITIALLY_SUSPENDED = FALSE
    AUTO_SUSPEND = 600
    AUTO_RESUME = TRUE
    MIN_CLUSTER_COUNT=1
    MAX_CLUSTER_COUNT=1
    SCALING_POLICY='STANDARD'
    COMMENT = '';

CREATE OR REPLACE NOTEBOOK FHIR_NB
    QUERY_WAREHOUSE=HL7_FHIR_WH
    MAIN_FILE = 'FHIR_NB.ipynb'
    WAREHOUSE=HL7_FHIR_WH;

Stage and access HL7 FHIR message. 

`Stages` are named storage locations in Snowflake used to temporarily load or unload data files before transferring them to Snowflake tables. A Snowflake stage essentially acts as a pointer or reference to a data file or set of data files. It enables Snowflake to access these data files for loading and unloading data without the need to copy or move the files.

In [ ]:
USE SCHEMA HL7_FHIR.HL7_FHIR_R4;

-- Directory tables store a catalog of staged files in cloud storage.
CREATE OR REPLACE STAGE HL7_FHIR_STAGE_INTERNAL
    DIRECTORY = (ENABLE = TRUE)
    COMMENT = 'Used for staging data';

In [ ]:
SHOW STAGES;

In [ ]:
-- DESCRIBE STAGE HL7_FHIR_STAGE_INTERNAL;

Stage data files from local file system by executing `PUT` command using `SnowSQL` client.

Invoking snowsql requires the Snowflake account identifier which can be retrieve through SnowSight UI or programatically as below.

In [ ]:
-- SELECT * FROM TABLE(FLATTEN(INPUT => PARSE_JSON(system$allowlist()))) T;

SELECT REPLACE(T.VALUE:host::VARCHAR, '.snowflakecomputing.com') AS ACCOUNT_IDENTIFIER
FROM TABLE(FLATTEN(INPUT => PARSE_JSON(system$allowlist()))) AS T
WHERE T.VALUE:type::VARCHAR = 'SNOWFLAKE_DEPLOYMENT_REGIONLESS';


In a terminal window set shell variables for account, username, role, database, schema and warehouse as per setup above, and then invoke snowsql.

`snowsql -a $SNOWSQL_ACCOUNT -u $SNOWSQL_USER -r $SNOWSQL_ROLE -d $SNOWSQL_DATABASE -s $SNOWSQL_SCHEMA -w $SNOWSQL_WAREHOUSE`

Enter user password when prompted then 

Execute PUT command to load HL7 FHIR data into stage.

`put file:///fhir/*.json @HL7_FHIR_STAGE_INTERNAL;`

557 Row(s) produced. Time Elapsed: 235.529s

Exit SnowSQL when done with CTRL+d;

In [ ]:
ALTER STAGE HL7_FHIR_STAGE_INTERNAL REFRESH;

SELECT COUNT(*) FROM DIRECTORY(@HL7_FHIR_STAGE_INTERNAL);



In [ ]:
SELECT * FROM DIRECTORY(@HL7_FHIR_STAGE_INTERNAL) LIMIT 5;


Load HL7 FHIR messages into Snowflake tables

In [ ]:
-- Create PATIENT table for HL7 data via JSON
CREATE TABLE HL7_FHIR.HL7_FHIR_R4.PATIENT
    (JSON_STRING VARIANT);

Use [JSON Editor Online](https://jsoneditoronline.org/) to view the structure of the HL7 FHIR JSON messages and facilitate creation of appropiate file format. 

A Snowflake `file format` is a named database object that encapsulates information about a data file. This information includes the file's type (CSV, JSON, etc.), formatting options, and compression method. Snowflake file formats are used to simplify the process of loading and unloading data from Snowflake tables.

In [ ]:
CREATE OR REPLACE FILE FORMAT HL7_FHIR.HL7_FHIR_R4.HL7_FHIR_JSON
    TYPE = 'JSON'
    COMPRESSION = 'AUTO'
    ENABLE_OCTAL = FALSE
    ALLOW_DUPLICATE = FALSE
    STRIP_OUTER_ARRAY = FALSE
    STRIP_NULL_VALUES = FALSE
    IGNORE_UTF8_ERRORS = FALSE;


In [ ]:
SHOW FILE FORMATS IN DATABASE HL7_FHIR;

In [ ]:
COPY INTO HL7_FHIR.HL7_FHIR_R4.PATIENT
FROM @HL7_FHIR_STAGE_INTERNAL
FILE_FORMAT = (FORMAT_NAME = 'HL7_FHIR_JSON')
ON_ERROR = 'SKIP_FILE';

In [ ]:
-- Select raw JSON
SELECT * FROM HL7_FHIR.HL7_FHIR_R4.PATIENT LIMIT 1;

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, flatten, round

session = get_active_session()

patient_df = session.table("HL7_FHIR.HL7_FHIR_R4.PATIENT")

In [ ]:
# Flatten entry array to get resources

resources = patient_df.select(flatten(patient_df.JSON_STRING, path="entry")).alias("entry").select(flatten(col("entry", "value"))) \
.select("SEQ", "KEY", "VALUE") \
.group_by("SEQ") \
.pivot("KEY", ['fullUrl', 'request', 'resource']).agg({'VALUE': "max"}).sort(col("SEQ").asc()) \
.with_column_renamed(col("'fullUrl'"), "fullUrl") \
.with_column_renamed(col("'request'"), "request") \
.with_column_renamed(col("'resource'"), "resource")



In [ ]:
# resources.columns
resources.first(10)
# resources.show(1)
# resources.limit(1).show()


In [ ]:
resources.select(col('request')['url'].cast('STRING').alias('resource_type')).distinct()

In [ ]:
patients = resources.filter(col('request')["url"] == "Patient") \
.select(col('resource')["id"].cast("STRING").alias('patient_id'), \
       col('resource')['identifier'][0]['value'].cast("STRING").alias('medical_record_number'), \
       col('resource')['name'][0]['family'].cast("STRING").alias('family_name'), \
       col('resource')['name'][0]['given'][0].cast("STRING").alias('given_name'), \
       col('resource')['gender'].cast("STRING").alias('gender'), \
       col('resource')['extension'][0]['extension'][0]['valueCoding']['display'].cast("STRING").alias('race'), \
       col('resource')['birthDate'].cast("DATE").alias('birth_date'),
       col('resource')['deceasedDateTime'].cast("TIMESTAMP").alias('deceased_datetime'), \
       col('resource')['address'][0]['city'].cast("STRING").alias('city'), \
       col('resource')['address'][0]['state'].cast("STRING").alias('state'), \
       col('resource')['address'][0]['postalCode'].cast("STRING").alias('postal_code'), \
       col('resource')['address'][0]['country'].cast("STRING").alias('country')
       )


In [ ]:
patients.first(10)


In [ ]:
# Get the schema
schema = patients.schema

# Print column names and types
for field in schema.fields:
    print(f"{field.name}: {field.datatype}")


In [ ]:
encounters = resources.filter(col('request')["url"] == "Encounter") \
.select(col('resource')["id"].cast("STRING").alias('encounter_id'), \
       col('resource')['subject']['reference'].substr(10, 50).alias('patient_id'), \
       col('resource')['status'].cast("STRING").alias('encounter_status'), \
       col('resource')['class']['code'].cast("STRING").alias('encounter_class_code'), \
       col('resource')['period']['start'].cast("TIMESTAMP").alias('encounter_start_datetime'), \
       col('resource')['period']['end'].cast("TIMESTAMP").alias('encounter_end_datetime'), \
       col('resource')['serviceProvider']['display'].cast("STRING").alias('encounter_service_provider_name'), \
       col('resource')['participant'][0]['individual']['display'].cast("STRING").alias('encounter_practitioner')
       )


In [ ]:
encounters.first(10)

In [ ]:
claims = resources.filter(col('request')["url"] == "Claim") \
.select(col('resource')["id"].cast("STRING").alias('claim_id'), \
       col('resource')['patient']['reference'].substr(10, 50).alias('patient_id'), \
       col('resource')['item'][0]['encounter'][0]['reference'].substring(10, 50).alias('encounter_id'), \
       col('resource')['status'].cast("STRING").alias('claim_status'), \
       col('resource')['type']['coding'][0]['code'].cast("STRING").alias('claim_type_code'), \
       col('resource')['total']['value'].cast("DOUBLE").alias('claim_amount'), \
       round(col('resource')['total']['value'], 2).alias('claim_amount_rounded'), \
       col('resource')['total']['currency'].cast("STRING").alias('claim_currency'), \
       col('resource')['created'].cast("TIMESTAMP").alias('claim_created_datetime'), \
       col('resource')['billablePeriod']['start'].cast("TIMESTAMP").alias('claim_start_datetime'), \
       col('resource')['billablePeriod']['end'].cast("TIMESTAMP").alias('claim_end_datetime'), \
       col('resource')['item'][0]['productOrService']['coding'][0]['code'].cast("STRING").alias('product_service_code'), \
       col('resource')['item'][0]['productOrService']['text'].cast("STRING").alias('product_service'), \
       col('resource')['insurance'][0]['coverage']['display'].cast("STRING").alias('insurer_name'), \
       col('resource')['provider']['display'].cast("STRING").alias('service_provider_name')
      )

In [ ]:
claims.first(10)

In [ ]:
conditions = resources.filter(col('request')['url'] == "Condition") \
.select(col('resource')["id"].cast("STRING").alias('condition_id'), \
       col('resource')['subject']['reference'].substr(10, 50).alias('patient_id'), \
       col('resource')['encounter']['reference'].substr(10, 50).alias('diagnosis_encounter_id'), \
       col('resource')['category'][0]['coding'][0]['code'].cast("STRING").alias('category'), \
       col('resource')['code']['coding'][0]['code'].cast("STRING").alias('diagnosis_code'), \
       col('resource')['code']['text'].cast("STRING").alias('diagnosis'), \
       col('resource')['clinicalStatus']['coding'][0]['code'].cast("STRING").alias('clinical_status'), \
       col('resource')['verificationStatus']['coding'][0]['code'].cast("STRING").alias('verification_status'), \
       col('resource')['recordedDate'].cast("TIMESTAMP").alias('diagnosis_datetime')
       )

In [ ]:
conditions.first(10)

In [ ]:
medications = resources.filter(col('request')['url'] == "MedicationRequest") \
.select(col('resource')["id"].cast("STRING").alias('medication_id'), \
       col('resource')['subject']['reference'].substr(10, 50).alias('patient_id'), \
       col('resource')['encounter']['reference'].substr(10, 50).alias('medication_encounter_id'), \
       col('resource')['medicationCodeableConcept']['coding'][0]['code'].cast("STRING").alias('medication_code'), \
       col('resource')['medicationCodeableConcept']['text'].cast("STRING").alias('medication'), \
       col('resource')['intent'].cast("STRING").alias('intent'), \
       col('resource')['status'].cast("STRING").alias('medication_status'), \
       col('resource')['authoredOn'].cast("TIMESTAMP").alias('medication_request_datetime'), \
       col('resource')['requester']['display'].cast("STRING").alias('medication_practitioner')
       )



In [ ]:
medications.first(10)

In [ ]:
claims_patients_medications_data = claims.join(patients, "patient_id", join_type = "left") \
    .join(encounters, ["patient_id", "encounter_id"], join_type = "left") \
    .join(medications, "patient_id", join_type= "left")
 
claims_patients_medications_data.select("patient_id", "encounter_id", "claim_id", "medication_id",
                                        "family_name", "given_name",
                                        "encounter_start_datetime", "encounter_end_datetime",
                                        "claim_amount", "claim_amount_rounded", "claim_status", "claim_type_code", "claim_created_datetime",
                                        "product_service_code", "product_service", 
                                        "service_provider_name", "insurer_name",
                                        "intent", 
                                        "medication_code", "medication", 
                                        "medication_request_datetime", "medication_status", "medication_practitioner"
                                       ).first(20)

In [ ]:
claims_patients_diagnoses_data = claims.join(patients, "patient_id", join_type = "left") \
    .join(encounters, ["patient_id", "encounter_id"], join_type = "left") \
    .join(conditions, "patient_id", join_type= "left")

claims_patients_diagnoses_data.select("patient_id", "encounter_id", "claim_id", "condition_id",
                                      "family_name", "given_name",
                                      "claim_amount", "claim_amount_rounded", "claim_status", "claim_type_code", "claim_created_datetime",
                                      "diagnosis_code", "diagnosis", "diagnosis_datetime").first(20)



In [ ]:
result = session.sql("create or replace schema HL7_FHIR.HARMONIZED").collect()
result


In [ ]:
patients.write.save_as_table("HL7_FHIR.HARMONIZED.PATIENTS", mode="overwrite", table_type="")
encounters.write.save_as_table("HL7_FHIR.HARMONIZED.ENCOUNTERS", mode="overwrite", table_type="")
claims.write.save_as_table("HL7_FHIR.HARMONIZED.CLAIMS", mode="overwrite", table_type="")
conditions.write.save_as_table("HL7_FHIR.HARMONIZED.CONDITIONS", mode="overwrite", table_type="")
medications.write.save_as_table("HL7_FHIR.HARMONIZED.MEDICATIONS", mode="overwrite", table_type="")




In [ ]:
session.table("HL7_FHIR.HARMONIZED.CLAIMS").show(10)


In [ ]:
CREATE OR REPLACE SCHEMA HL7_FHIR.ANALYTICS;

In [ ]:
USE ROLE ACCOUNTADMIN;
USE DATABASE HL7_FHIR;
USE SCHEMA HARMONIZED;

In [ ]:
CREATE OR REPLACE STAGE HL7_FHIR.HARMONIZED.CORTEX_ANALYST_STAGE 
    DIRECTORY = (ENABLE = TRUE);

Construct a __`Semantic Model`__, __`hl7_fhir_r4_semantic_model.yaml`__, that captures relationships between PATIENTS, ENCOUNTERS, CLAIMS, CONDITIONS, MEDICATIONS tables for use by Cortex Analyst.

The Semantic Model is composed of a number of different fields that help Cortex Analyst understand the specifics of data:

- Logical Tables which are composed of Logical Columns
- Logical Columns which are one of dimensions, time_dimensions, or measures
- Relationships that exist between tables to allow for JOINS
- Verifies queries - a collection of questions and corresponding SQL queries to answer the questions that can help improve accuracy and trustworthiness of results. Used by Cortex Analyst when answering similar types of questions.

Go To AI & ML Studio and choose Cortex Analyst to create and refine the semantic model.

Save the semantic model to CORTEX_ANALYST_STAGE.

Cortex Analyst 

- Answer to explain the dataset

This semantic data model contains healthcare data including patient information, medical encounters, conditions/diagnoses, medications, and insurance claims. It allows analysis of patient demographics, medical histories, diagnoses trends, medication prescriptions, and insurance claim patterns. You can analyze things like claim amounts by provider, common diagnoses by patient demographics, medication prescribing patterns, and encounter volumes over time. The data model connects patients to their medical encounters, conditions, medications and related insurance claims to provide a comprehensive view of healthcare delivery and costs.

- Suggested questions

    - What is the total claim amount for institutional claims in the last month?
    - How many unique patients have received tramadol hydrochloride 50 MG Oral Tablet medication year to date?
    - What is the average length of stay (in days) for encounters at HEBREW REHABILITATION CENTER?

Create __`Streamlit Chat App`__ to call the __`Cortex Analyst REST API`__ with the __`configured semantic model`__. The conversational app allows users to interact with HL7 FHIR data using natural language question-asking.

The Chat App interface include a description of the semantic data model, onboarded verified questions, and a chat input to process user input prompts by sending POST requests to the specified Cortex Analyst API endpoint, and update conversation history between user and AI analyst.

The Chat App interface presents the SQL query executed by the assistant along with the results in form of Pandas dataframes and charts.

Semantic Model Description from Cortex Analyst in the chat interface: This semantic data model contains healthcare data including insurance claims, patient encounters, medical conditions, medications, and patient demographics. You can analyze things like claim amounts, diagnoses, medications prescribed, patient visit details, and patient characteristics. For example, you could look at total claim amounts by diagnosis, track patient visits over time, analyze medication prescriptions by provider, or examine patient demographics. The data allows for both financial analysis of healthcare costs as well as clinical analysis of diagnoses, treatment and patient care patterns.

💡✨🧠 __Analytics__

Ask questions to analyze healthcare data with Cortex Analyst.

📊 __Clinical Utilization & Care Pathways__

Q1. Which patients have the highest frequency of encounters and associated costs? 

Q2. Which patients have unusually high frequency of encounters and medications aimed at detecting overutilization of services? 

Q3. Which are the patients with multi-morbidity and polypharmacy risk, that is patients with ≥3 chronic conditions and ≥5 unique medications? Chronic conditions include diabetes, diabetic renal disease, hypertension, coronary heart disease, and stroke.

Q4. Find patients with chronic conditions but low medication adherence 

Q5. Which conditions most frequently lead to medication claims? 

In [ ]:
-- Which patients have the highest frequency of encounters and associated costs?

/*Name: patients with the highest frequency of encounters and associated costs
Question: Which patients have the highest frequency of encounters and associated costs?
Verified by: Lavinia Carabet
Verified at: 2025-06-05 16:57:36
SQL query:

SELECT p.patient_id, COUNT(DISTINCT e.encounter_id) AS num_encounters, SUM(c.claim_amount) AS total_claim_amount FROM __patients AS p JOIN __encounters AS e ON p.patient_id = e.patient_id JOIN __claims AS c ON e.encounter_id = c.encounter_id GROUP BY p.patient_id ORDER BY num_encounters DESC, total_claim_amount DESC LIMIT 10*/ -- in Streamlit app

SELECT 
    p.patient_id,
    COUNT(DISTINCT e.encounter_id) AS num_encounters,
    SUM(c.claim_amount) AS total_claim_amount
FROM PATIENTS p
JOIN ENCOUNTERS e ON p.patient_id = e.patient_id
JOIN CLAIMS c ON e.encounter_id = c.encounter_id
GROUP BY p.patient_id
ORDER BY num_encounters DESC, total_claim_amount DESC
LIMIT 10;

In [ ]:
-- Which patients have the highest frequency of encounters and associated costs?

WITH __patients AS (
  SELECT
    patient_id
  FROM hl7_fhir.harmonized.patients
), __encounters AS (
  SELECT
    encounter_id,
    patient_id
  FROM hl7_fhir.harmonized.encounters
), __claims AS (
  SELECT
    encounter_id,
    claim_amount
  FROM hl7_fhir.harmonized.claims
)
SELECT
  p.patient_id,
  COUNT(DISTINCT e.encounter_id) AS num_encounters,
  SUM(c.claim_amount) AS total_claim_amount
FROM __patients AS p
JOIN __encounters AS e
  ON p.patient_id = e.patient_id
JOIN __claims AS c
  ON e.encounter_id = c.encounter_id
GROUP BY
  p.patient_id
ORDER BY
  num_encounters DESC,
  total_claim_amount DESC
LIMIT 10
 -- Generated by Cortex Analyst
;

In [ ]:
-- detect overutilization: patients with unusually high frequency of encounters and medications

WITH encounter_counts AS (
    SELECT patient_id, COUNT(DISTINCT encounter_id) AS encounter_count
    FROM ENCOUNTERS
    GROUP BY patient_id
),
medication_counts AS (
    SELECT patient_id, COUNT(DISTINCT medication_id) AS medication_count
    FROM MEDICATIONS
    GROUP BY patient_id
),
claim_counts AS (
    SELECT patient_id, COUNT(DISTINCT claim_id) AS claim_count
    FROM CLAIMS
    GROUP BY patient_id
)

SELECT 
    p.patient_id,
    p.family_name,
    p.given_name,
    p.birth_date,
    p.deceased_datetime,
    p.city || ' ' || p.state || ' ' || p.postal_code as address,
    COALESCE(e.encounter_count, 0) AS encounters,
    COALESCE(m.medication_count, 0) AS meds,
    COALESCE(c.claim_count, 0) AS claims
FROM PATIENTS p
LEFT JOIN encounter_counts e ON p.patient_id = e.patient_id
LEFT JOIN medication_counts m ON p.patient_id = m.patient_id
LEFT JOIN claim_counts c ON p.patient_id = c.patient_id
WHERE COALESCE(e.encounter_count, 0) > 20
  AND COALESCE(m.medication_count, 0) > 15
ORDER BY encounters DESC, meds DESC;


In [ ]:
/*Which patients have unusually high frequency of encounters and medications aimed at detecting overutilization of services?

Interpretation: Identify patients who show potential overutilization of healthcare services by finding those with both high numbers of medical encounters and high numbers of medication prescriptions over the entire available time period. We'll consider this by looking at the total count of encounters and medications per patient, and rank them to identify outliers.*/

WITH __patients AS (
  SELECT
    patient_id,
    family_name,
    given_name
  FROM hl7_fhir.harmonized.patients
), __encounters AS (
  SELECT
    encounter_id,
    patient_id
  FROM hl7_fhir.harmonized.encounters
), __medications AS (
  SELECT
    medication_id,
    patient_id
  FROM hl7_fhir.harmonized.medications
), patient_encounters AS (
  SELECT
    p.patient_id,
    COUNT(DISTINCT e.encounter_id) AS num_encounters
  FROM __patients AS p
  LEFT OUTER JOIN __encounters AS e
    ON p.patient_id = e.patient_id
  GROUP BY
    p.patient_id
), patient_medications AS (
  SELECT
    p.patient_id,
    COUNT(DISTINCT m.medication_id) AS num_medications
  FROM __patients AS p
  LEFT OUTER JOIN __medications AS m
    ON p.patient_id = m.patient_id
  GROUP BY
    p.patient_id
), utilization_metrics AS (
  SELECT
    pe.patient_id,
    pe.num_encounters,
    pm.num_medications,
    PERCENT_RANK() OVER (ORDER BY pe.num_encounters) AS encounter_percentile,
    PERCENT_RANK() OVER (ORDER BY pm.num_medications) AS medication_percentile
  FROM patient_encounters AS pe
  JOIN patient_medications AS pm
    ON pe.patient_id = pm.patient_id
)
SELECT
  um.patient_id,
  p.family_name,
  p.given_name,
  um.num_encounters,
  um.num_medications
FROM utilization_metrics AS um
LEFT OUTER JOIN __patients AS p
  ON um.patient_id = p.patient_id
WHERE
  um.encounter_percentile >= 0.9 AND um.medication_percentile >= 0.9
ORDER BY
  um.num_encounters DESC NULLS LAST,
  um.num_medications DESC NULLS LAST
 -- Generated by Cortex Analyst
;

In [ ]:
/*
Which patients have unusually high frequency of encounters and medications aimed at detecting overutilization of services? include total number of claims

Interpretation: Find patients who potentially overutilize healthcare services by analyzing their service usage patterns over the entire available time period. Specifically, calculate for each patient: 1) total number of medical encounters, 2) total number of unique medication prescriptions, and 3) total number of claims. Include patient details (ID, gender, city, state) along with these metrics, ordered by total encounters in descending order to identify high utilizers.

PROBLEM: Running time of the query: 2m 26s
*/

WITH __patients AS (
  SELECT
    patient_id,
    gender,
    city,
    state
  FROM hl7_fhir.harmonized.patients
), __encounters AS (
  SELECT
    encounter_id,
    patient_id
  FROM hl7_fhir.harmonized.encounters
), __medications AS (
  SELECT
    medication_id,
    patient_id
  FROM hl7_fhir.harmonized.medications
), __claims AS (
  SELECT
    claim_id,
    patient_id
  FROM hl7_fhir.harmonized.claims
), patient_metrics AS (
  SELECT
    p.patient_id,
    COUNT(DISTINCT e.encounter_id) AS num_encounters,
    COUNT(DISTINCT m.medication_id) AS num_medications,
    COUNT(DISTINCT c.claim_id) AS num_claims
  FROM __patients AS p
  LEFT OUTER JOIN __encounters AS e
    ON p.patient_id = e.patient_id
  LEFT OUTER JOIN __medications AS m
    ON p.patient_id = m.patient_id
  LEFT OUTER JOIN __claims AS c
    ON p.patient_id = c.patient_id
  GROUP BY
    p.patient_id
)
SELECT
  p.patient_id,
  p.gender,
  p.city,
  p.state,
  pm.num_encounters,
  pm.num_medications,
  pm.num_claims
FROM __patients AS p
JOIN patient_metrics AS pm
  ON p.patient_id = pm.patient_id
WHERE
  pm.num_encounters > 0 OR pm.num_medications > 0 OR pm.num_claims > 0
ORDER BY
  pm.num_encounters DESC NULLS LAST,
  pm.num_medications DESC NULLS LAST,
  pm.num_claims DESC NULLS LAST
 -- Generated by Cortex Analyst
;

PROBLEM: Running time of the query: 2m 26s

SLOW PERFORMANCE DUE TO:

1. Multiple (three) Large LEFT JOINS in One Pass - This causes a huge intermediate result set, especially when patients have many encounters, medications, and claims — causing explosion in row count due to Cartesian effect in left joins (e.g., 10 encounters × 10 medications × 10 claims = 1,000 rows for one patient).

2. COUNT(DISTINCT ...) Over Duplicated Joins - The join explosion makes COUNT(DISTINCT ...) much more expensive, as Snowflake must deduplicate large intermediate sets.

3. Unnecessary Rejoin - __patients is joined twice: once in patient_metrics, again in the final SELECT. That adds a redundant join.

Rewrite with Pre-Aggregated Metrics Before Joining

Split the logic into independent per-table aggregations, then join the small aggregated sets:

Running time: < 1s

WHY > 120X PERFORMANCE GAIN:

1. Each subquery (encounter_counts, etc.) runs independently and efficiently via GROUP BY, producing small result sets (1 row per patient).

2. Final join is on patient_id only, with minimal duplication.

3. COUNT(DISTINCT) is performed before any joins.


In [ ]:
WITH encounter_counts AS (
  SELECT patient_id, COUNT(DISTINCT encounter_id) AS num_encounters
  FROM hl7_fhir.harmonized.encounters
  GROUP BY patient_id
),
medication_counts AS (
  SELECT patient_id, COUNT(DISTINCT medication_id) AS num_medications
  FROM hl7_fhir.harmonized.medications
  GROUP BY patient_id
),
claim_counts AS (
  SELECT patient_id, COUNT(DISTINCT claim_id) AS num_claims
  FROM hl7_fhir.harmonized.claims
  GROUP BY patient_id
)

SELECT
  p.patient_id,
  p.gender,
  p.race,
  p.birth_date,
  p.deceased_datetime,
  p.city,
  p.state,
  p.postal_code,
  COALESCE(e.num_encounters, 0) AS num_encounters,
  COALESCE(m.num_medications, 0) AS num_medications,
  COALESCE(c.num_claims, 0) AS num_claims
FROM hl7_fhir.harmonized.patients p
LEFT JOIN encounter_counts e ON p.patient_id = e.patient_id
LEFT JOIN medication_counts m ON p.patient_id = m.patient_id
LEFT JOIN claim_counts c ON p.patient_id = c.patient_id
WHERE COALESCE(e.num_encounters, 0) > 0
   OR COALESCE(m.num_medications, 0) > 0
   OR COALESCE(c.num_claims, 0) > 0
ORDER BY
  num_encounters DESC NULLS LAST,
  num_medications DESC NULLS LAST,
  num_claims DESC NULLS LAST;



Refactor queries into Snowflake materialized views, giving persistent, pre-aggregated data for fast downstream queries and read-heavy workloads (dashboards, reports). Snowflake keeps the view updated automatically when base tables change (with some latency).

Materialized views store precomputed results as physical tables, which significantly boosts query performance. By reducing the need for repetitive calculations, materialized views optimize resource usage and improve end-user response times.


In [ ]:
CREATE OR REPLACE TABLE HL7_FHIR.ANALYTICS.patient_utilization_of_services_snapshot AS
WITH encounter_counts AS (
    SELECT patient_id, COUNT(DISTINCT encounter_id) AS encounter_count
    FROM ENCOUNTERS
    GROUP BY patient_id
),
medication_counts AS (
    SELECT patient_id, COUNT(DISTINCT medication_id) AS medication_count
    FROM MEDICATIONS
    GROUP BY patient_id
),
claim_counts AS (
    SELECT patient_id, COUNT(DISTINCT claim_id) AS claim_count
    FROM CLAIMS
    GROUP BY patient_id
)

SELECT 
    p.patient_id,
    p.family_name,
    p.given_name,
    p.gender,
    p.race,
    p.city,
    p.state,
    p.postal_code,
    p.birth_date, 
    DATE(p.deceased_datetime) as deceased_date,
    COALESCE(e.encounter_count, 0) AS num_encounters,
    COALESCE(m.medication_count, 0) AS num_medications,
    COALESCE(c.claim_count, 0) AS num_claims
FROM PATIENTS p
LEFT JOIN encounter_counts e ON p.patient_id = e.patient_id
LEFT JOIN medication_counts m ON p.patient_id = m.patient_id
LEFT JOIN claim_counts c ON p.patient_id = c.patient_id
ORDER BY num_encounters DESC, num_medications DESC;

In [ ]:
CREATE OR REPLACE MATERIALIZED VIEW HL7_FHIR.ANALYTICS.patient_utilization_of_services_mv AS
SELECT *
FROM HL7_FHIR.ANALYTICS.patient_utilization_of_services_snapshot;

Add index clustering to speed up point lookups or filters on num_encounters, num_medications, or num_claims. 

This helps performance at scale and if these columns are queried frequently.




In [ ]:
ALTER MATERIALIZED VIEW HL7_FHIR.ANALYTICS.patient_utilization_of_services_mv 
CLUSTER BY (num_encounters, num_medications, num_claims);


In [ ]:
-- OVERUTILIZATION OF SERVICES

SELECT * FROM HL7_FHIR.ANALYTICS.patient_utilization_of_services_mv
WHERE num_encounters > 20 AND num_medications > 15
ORDER BY num_encounters DESC, num_medications DESC;

Using the fixed/optimized Cortex generated Q2 query, create independent materialized views per metric and a final view that joins them for reporting, an approach that provides better performance, lower refresh cost, and maximum reusability.

Performance & Benefits

- Fast joins - Materialized views already have one row per patient_id             
- Low refresh cost - Each MV refreshes only when its source table changes         
- Scalable - Can add more metrics (e.g., lab results, visits) as separate MVs 
- Reusable - Metrics are decoupled — usable in other dashboards/views too         

Limitations

Snowflake restricts materialized views to ensure they can be incrementally and efficiently refreshed, and COUNT(DISTINCT ...) is not incrementally refreshable due to its global deduplication semantics.

Replacement of COUNT(DISTINCT ...) with APPROX_COUNT_DISTINCT(...) allowed in materialized views, gives approximate counts, typically > 99% accurate — often enough for analytics on large datasets where a small margin of error is acceptable.

Alternatives

- Use tables to materialize individual metrics when accuracy matters and materialization is wanted
- Use regular views (non-materialized) when workload allows slower queries and in design phase

In [ ]:
CREATE OR REPLACE MATERIALIZED VIEW HL7_FHIR.ANALYTICS.patient_encounter_counts_mv AS
SELECT
  patient_id,
  APPROX_COUNT_DISTINCT(encounter_id) AS num_encounters
FROM hl7_fhir.harmonized.encounters
GROUP BY patient_id;


In [ ]:
CREATE OR REPLACE MATERIALIZED VIEW HL7_FHIR.ANALYTICS.patient_medication_counts_mv AS
SELECT
  patient_id,
  APPROX_COUNT_DISTINCT(medication_id) AS num_medications
FROM hl7_fhir.harmonized.medications
GROUP BY patient_id;


In [ ]:
CREATE OR REPLACE MATERIALIZED VIEW HL7_FHIR.ANALYTICS.patient_claim_counts_mv AS
SELECT
  patient_id,
  APPROX_COUNT_DISTINCT(claim_id) AS num_claims  
FROM hl7_fhir.harmonized.claims
GROUP BY patient_id;


In [ ]:
CREATE OR REPLACE VIEW HL7_FHIR.ANALYTICS.patient_utilization_of_services_v AS
SELECT
    p.patient_id,
    p.family_name,
    p.given_name,
    p.gender,
    p.race,
    p.city,
    p.postal_code,
    p.state,
    p.birth_date, 
    DATE(p.deceased_datetime) as deceased_date,
    COALESCE(e.num_encounters, 0) AS num_encounters,
    COALESCE(m.num_medications, 0) AS num_medications,
    COALESCE(c.num_claims, 0) AS num_claims
FROM hl7_fhir.harmonized.patients p
LEFT JOIN HL7_FHIR.ANALYTICS.patient_encounter_counts_mv e
  ON p.patient_id = e.patient_id
LEFT JOIN HL7_FHIR.ANALYTICS.patient_medication_counts_mv m
  ON p.patient_id = m.patient_id
LEFT JOIN HL7_FHIR.ANALYTICS.patient_claim_counts_mv c
  ON p.patient_id = c.patient_id
WHERE COALESCE(e.num_encounters, 0) > 0
   OR COALESCE(m.num_medications, 0) > 0
   OR COALESCE(c.num_claims, 0) > 0
ORDER BY
  num_encounters DESC NULLS LAST,
  num_medications DESC NULLS LAST,
  num_claims DESC NULLS LAST;


In [ ]:
SELECT * FROM HL7_FHIR.ANALYTICS.patient_utilization_of_services_v
WHERE num_encounters > 20 AND num_medications > 15
ORDER BY num_encounters DESC, num_medications DESC;


In [ ]:
-- Patients with multi-morbidity and polypharmacy risk: patients with ≥3 chronic conditions and ≥5 unique medications.

-- Common chronic: 'diabetes', 'diabetic renal disease', 'hypertension', 'coronary heart disease', 'stroke'

WITH condition_counts AS (
    SELECT patient_id, COUNT(DISTINCT diagnosis_code) AS chronic_conditions
    FROM CONDITIONS
    WHERE diagnosis_code IN ('44054006', '127013003', '59621000', '53741008', '230690007') -- common chronic
    GROUP BY patient_id
),
med_counts AS (
    SELECT patient_id, COUNT(DISTINCT medication_code) AS med_count
    FROM MEDICATIONS
    WHERE medication_status = 'active'
    GROUP BY patient_id
)
SELECT 
    p.patient_id,
    p.family_name,
    p.given_name,
    p.city || ' ' || p.state || ' ' || p.postal_code as address,
    chronic_conditions,
    med_count
FROM condition_counts cc
JOIN med_counts mc ON cc.patient_id = mc.patient_id
JOIN PATIENTS p ON p.patient_id = cc.patient_id
WHERE chronic_conditions >= 3 AND med_count >= 5
ORDER BY chronic_conditions DESC, med_count DESC;


In [ ]:
/*
Which are the patients with multi-morbidity and polypharmacy risk, that is patients with ≥3 chronic conditions and ≥5 unique medications? Chronic conditions include diabetes, diabetic renal disease, hypertension, coronary heart disease, and stroke.

Find patients who have both multi-morbidity (3 or more chronic conditions from the list: diabetes, diabetic renal disease, hypertension, coronary heart disease, and stroke) and polypharmacy risk (5 or more unique medications) over their entire medical history.

- WRONG DIAGNOSIS_CODE FOR HYPERTENSION IN CORTEX ANALYST ANSWER: IS ACTUALLY '59621000' NOT '38341003'
*/

WITH __patients AS (
  SELECT
    patient_id,
    family_name,
    given_name
  FROM hl7_fhir.harmonized.patients
), __conditions AS (
  SELECT
    patient_id,
    diagnosis_code,
    verification_status
  FROM hl7_fhir.harmonized.conditions
), __medications AS (
  SELECT
    patient_id,
    medication_code,
    medication_status
  FROM hl7_fhir.harmonized.medications
), chronic_conditions AS (
  SELECT
    c.patient_id,
    COUNT(DISTINCT c.diagnosis_code) AS num_conditions
  FROM __conditions AS c
  WHERE
    c.diagnosis_code IN ('44054006', '127013003', '38341003', '53741008', '230690007')
    AND c.verification_status = 'confirmed'
  GROUP BY
    c.patient_id
  HAVING
    num_conditions >= 3
), medication_counts AS (
  SELECT
    m.patient_id,
    COUNT(DISTINCT m.medication_code) AS num_medications
  FROM __medications AS m
  WHERE
    m.medication_status = 'active'
  GROUP BY
    m.patient_id
  HAVING
    num_medications >= 5
)
SELECT
  p.patient_id,
  p.family_name,
  p.given_name,
  cc.num_conditions,
  mc.num_medications
FROM __patients AS p
JOIN chronic_conditions AS cc
  ON p.patient_id = cc.patient_id
JOIN medication_counts AS mc
  ON p.patient_id = mc.patient_id
ORDER BY
  cc.num_conditions DESC NULLS LAST,
  mc.num_medications DESC NULLS LAST
 -- Generated by Cortex Analyst
;

In [ ]:
WITH __patients AS (
  SELECT
    patient_id,
    family_name,
    given_name
  FROM hl7_fhir.harmonized.patients
), __conditions AS (
  SELECT
    patient_id,
    diagnosis_code,
    verification_status
  FROM hl7_fhir.harmonized.conditions
), __medications AS (
  SELECT
    patient_id,
    medication_code,
    medication_status
  FROM hl7_fhir.harmonized.medications
), chronic_conditions AS (
  SELECT
    c.patient_id,
    COUNT(DISTINCT c.diagnosis_code) AS num_conditions
  FROM __conditions AS c
  WHERE
    c.diagnosis_code IN ('44054006', '127013003', '59621000', '53741008', '230690007')
    AND c.verification_status = 'confirmed'
  GROUP BY
    c.patient_id
  HAVING
    num_conditions >= 3
), medication_counts AS (
  SELECT
    m.patient_id,
    COUNT(DISTINCT m.medication_code) AS num_medications
  FROM __medications AS m
  WHERE
    m.medication_status = 'active'
  GROUP BY
    m.patient_id
  HAVING
    num_medications >= 5
)
SELECT
  p.patient_id,
  p.family_name,
  p.given_name,
  cc.num_conditions,
  mc.num_medications
FROM __patients AS p
JOIN chronic_conditions AS cc
  ON p.patient_id = cc.patient_id
JOIN medication_counts AS mc
  ON p.patient_id = mc.patient_id
ORDER BY
  cc.num_conditions DESC NULLS LAST,
  mc.num_medications DESC NULLS LAST
 -- Generated by Cortex Analyst
;

In [ ]:
-- Find patients with chronic conditions but low medication adherence
-- Use a ratio of days covered or gaps in refill history

WITH med_windows AS (
    SELECT 
        patient_id,        
        MIN(DATE(medication_request_datetime)) AS first_fill,
        MAX(DATE(medication_request_datetime)) AS last_fill,
        COUNT(DISTINCT medication_request_datetime) AS num_fills
    FROM MEDICATIONS
    GROUP BY patient_id
), 
chronic_conditions AS (
    SELECT patient_id, diagnosis_code
    FROM CONDITIONS
    WHERE diagnosis_code IN ('44054006', '59621000') -- diabetes, hypertension
    GROUP BY patient_id, diagnosis_code
)

SELECT 
    mw.patient_id,
    diagnosis_code,
    DATEDIFF('day', first_fill, last_fill) AS duration_days,
    FIRST_FILL,
    LAST_FILL,
    num_fills,
    ROUND(num_fills * 30.0 / NULLIF(DATEDIFF('day', first_fill, last_fill), 0), 2) AS adherence_ratio
FROM med_windows mw
JOIN chronic_conditions cc ON cc.patient_id = mw.patient_id
WHERE num_fills > 1
ORDER BY adherence_ratio ASC
LIMIT 50;


In [ ]:
-- conditions that most frequently lead to medication claims

SELECT 
    c.diagnosis_code,
    c.diagnosis,
    COUNT(DISTINCT m.medication_id) AS medication_count
FROM CONDITIONS c
JOIN MEDICATIONS m ON c.patient_id = m.patient_id
GROUP BY c.diagnosis_code, c.diagnosis
ORDER BY medication_count DESC
LIMIT 40;


💰 __Claims and Financial Insights__

Q6. How do claim amounts vary by medication prescribed? 

Q7. What is the yearly trend of total claim amount per top condition?

Q8. Which conditions are associated with the highest claim amounts?

Q9. What are the most frequently claimed medications by condition type? 

Q10. Find cost per condition per patient as an efficiency and burden measure. 

Q11. What is the yearly trend of total claim amount per claim type? 

Q12. What is the yearly trend of total claim amount per diagnosis per service provider? 

In [ ]:
/*
Name: claim amount variability by medication
Question: How do claim amounts vary by medication prescribed?
Verified by: Lavinia Carabet
Verified at: 2025-06-05 17:03:22
SQL query:

SELECT medication_code, MIN(medication) AS medication, AVG(claim_amount) AS avg_claim, SUM(claim_amount) AS total_claim FROM __medications AS m JOIN __patients AS p ON p.patient_id = m.patient_id JOIN __claims AS c ON c.patient_id = p.patient_id WHERE NOT medication_code IS NULL GROUP BY medication_code ORDER BY total_claim DESC LIMIT 20
*/

SELECT 
    medication_code,
    MIN(medication) AS medication,
    AVG(claim_amount) AS avg_claim,
    SUM(claim_amount) AS total_claim
FROM MEDICATIONS m
JOIN PATIENTS p ON p.patient_id = m.patient_id
JOIN CLAIMS c ON c.patient_id = p.patient_id
WHERE medication_code IS NOT NULL
GROUP BY medication_code
ORDER BY total_claim DESC
LIMIT 20;

In [ ]:
WITH __medications AS (
  SELECT
    patient_id,
    medication_code,
    medication
  FROM hl7_fhir.harmonized.medications
), __patients AS (
  SELECT
    patient_id
  FROM hl7_fhir.harmonized.patients
), __claims AS (
  SELECT
    patient_id,
    claim_amount
  FROM hl7_fhir.harmonized.claims
)
SELECT
  medication_code,
  MIN(medication) AS medication,
  AVG(claim_amount) AS avg_claim,
  SUM(claim_amount) AS total_claim
FROM __medications AS m
JOIN __patients AS p
  ON p.patient_id = m.patient_id
JOIN __claims AS c
  ON c.patient_id = p.patient_id
WHERE
  NOT medication_code IS NULL
GROUP BY
  medication_code
ORDER BY
  total_claim DESC
LIMIT 20
 -- Generated by Cortex Analyst
;

In [ ]:
-- Time-series: Yearly trend of total claim amount per top condition

SELECT 
    DATE_TRUNC('year', DATE(cl.claim_created_datetime)) AS claim_year,
    c.diagnosis_code,
    MIN(c.diagnosis) AS diagnosis,
    SUM(cl.claim_amount) AS total_claims
FROM CLAIMS cl
JOIN CONDITIONS c ON cl.patient_id = c.patient_id
WHERE c.diagnosis_code IN (
    SELECT diagnosis_code
    FROM CONDITIONS
    GROUP BY diagnosis_code
    ORDER BY COUNT(*) DESC
    LIMIT 5
)
GROUP BY claim_year, c.diagnosis_code
ORDER BY claim_year DESC, total_claims DESC;


In [ ]:
/*
What is the yearly trend of total claim amount per top condition?

Interpretation: What is the yearly trend of total claim amount for the top 10 conditions (by total claim amount) over the entire available time period? Note: A condition is identified by its diagnosis code.
*/

WITH __conditions AS (
  SELECT
    patient_id,
    diagnosis_code,
    diagnosis
  FROM hl7_fhir.harmonized.conditions
), __patients AS (
  SELECT
    patient_id
  FROM hl7_fhir.harmonized.patients
), __claims AS (
  SELECT
    patient_id,
    claim_created_datetime,
    claim_amount
  FROM hl7_fhir.harmonized.claims
), condition_totals AS (
  SELECT
    d.diagnosis_code,
    d.diagnosis,
    SUM(c.claim_amount) AS total_claims
  FROM __conditions AS d
  JOIN __patients AS p
    ON p.patient_id = d.patient_id
  JOIN __claims AS c
    ON c.patient_id = p.patient_id
  GROUP BY
    d.diagnosis_code,
    d.diagnosis
  ORDER BY
    total_claims DESC NULLS LAST
  LIMIT 10
), yearly_claims AS (
  SELECT
    DATE_PART('year', c.claim_created_datetime) AS claim_year,
    d.diagnosis_code,
    d.diagnosis,
    SUM(c.claim_amount) AS yearly_claim_amount
  FROM __conditions AS d
  JOIN __patients AS p
    ON p.patient_id = d.patient_id
  JOIN __claims AS c
    ON c.patient_id = p.patient_id
  WHERE
    d.diagnosis_code IN (
      SELECT
        diagnosis_code
      FROM condition_totals
    )
  GROUP BY
    claim_year,
    d.diagnosis_code,
    d.diagnosis
)
SELECT
  y.claim_year,
  y.diagnosis_code,
  y.diagnosis,
  y.yearly_claim_amount
FROM yearly_claims AS y
ORDER BY
  y.claim_year DESC NULLS LAST,
  y.yearly_claim_amount DESC NULLS LAST
 -- Generated by Cortex Analyst
;

In [ ]:
-- conditions associated with the highest claim amounts

SELECT 
    c.diagnosis_code,
    MIN(c.diagnosis) AS diagnosis,    
    MIN(c.clinical_status) AS clinical_status,
    SUM(cl.claim_amount) AS total_claim
FROM CONDITIONS c
JOIN PATIENTS p ON p.patient_id = c.patient_id
JOIN CLAIMS cl ON p.patient_id = cl.patient_id
GROUP BY c.diagnosis_code
ORDER BY total_claim DESC
LIMIT 20;


In [ ]:
/*
Which conditions are associated with the highest claim amounts?

Interpretation: What are the medical conditions that are associated with the highest average and total claim amounts over the entire available time period? This will help identify which conditions typically result in the most expensive claims.
*/

WITH __conditions AS (
  SELECT
    patient_id,
    diagnosis_code,
    diagnosis
  FROM hl7_fhir.harmonized.conditions
), __patients AS (
  SELECT
    patient_id
  FROM hl7_fhir.harmonized.patients
), __claims AS (
  SELECT
    patient_id,
    claim_amount
  FROM hl7_fhir.harmonized.claims
)
SELECT
  c.diagnosis_code,
  MIN(c.diagnosis) AS condition_name,
  AVG(cl.claim_amount) AS avg_claim_amount,
  SUM(cl.claim_amount) AS total_claim_amount
FROM __conditions AS c
JOIN __patients AS p
  ON p.patient_id = c.patient_id
JOIN __claims AS cl
  ON cl.patient_id = p.patient_id
WHERE
  NOT c.diagnosis_code IS NULL
GROUP BY
  c.diagnosis_code
ORDER BY
  total_claim_amount DESC NULLS LAST
LIMIT 20
 -- Generated by Cortex Analyst
;

In [ ]:
-- most frequently claimed medications by condition type

SELECT 
    c.diagnosis_code,
    MIN(c.diagnosis) AS diagnosis,
    m.medication_code,
    MIN(m.medication) AS medication,
    COUNT(*) AS claim_count
FROM CONDITIONS c
JOIN MEDICATIONS m ON c.patient_id = m.patient_id
GROUP BY c.diagnosis_code, m.medication_code
ORDER BY claim_count DESC
LIMIT 20;


In [ ]:
/*
What are the most frequently claimed medications by condition type?

Interpretation: What are the most common medical conditions and their associated medications based on the number of claims filed? This will show us which medications are most frequently claimed for each type of medical condition.
*/

WITH __conditions AS (
  SELECT
    patient_id,
    diagnosis_code,
    diagnosis,
    verification_status
  FROM hl7_fhir.harmonized.conditions
), __patients AS (
  SELECT
    patient_id
  FROM hl7_fhir.harmonized.patients
), __medications AS (
  SELECT
    patient_id,
    medication_code,
    medication
  FROM hl7_fhir.harmonized.medications
), __claims AS (
  SELECT
    claim_id,
    patient_id
  FROM hl7_fhir.harmonized.claims
), condition_meds AS (
  SELECT
    c.diagnosis_code,
    c.diagnosis,
    m.medication_code,
    m.medication,
    COUNT(DISTINCT cl.claim_id) AS claim_count
  FROM __conditions AS c
  JOIN __patients AS p
    ON p.patient_id = c.patient_id
  JOIN __medications AS m
    ON m.patient_id = p.patient_id
  JOIN __claims AS cl
    ON cl.patient_id = p.patient_id
  WHERE
    c.verification_status = 'confirmed'
    AND NOT c.diagnosis_code IS NULL
    AND NOT m.medication_code IS NULL
  GROUP BY
    c.diagnosis_code,
    c.diagnosis,
    m.medication_code,
    m.medication
)
SELECT
  diagnosis_code,
  diagnosis,
  medication_code,
  medication,
  claim_count,
  RANK() OVER (PARTITION BY diagnosis_code ORDER BY claim_count DESC NULLS LAST) AS rank_within_condition
FROM condition_meds
ORDER BY
  claim_count DESC NULLS LAST
 -- Generated by Cortex Analyst
;

In [ ]:
-- cost per condition per patient — efficiency and burden measure

SELECT 
    cl.patient_id,
    diagnosis_code,
    MIN(diagnosis) AS diagnosis,
    SUM(claim_amount) AS total_condition_cost,
    COUNT(DISTINCT claim_id) AS claims_count,
    ROUND(SUM(claim_amount) / NULLIF(COUNT(DISTINCT claim_id), 0), 2) AS avg_cost_per_claim
FROM CONDITIONS c
JOIN CLAIMS cl ON c.patient_id = cl.patient_id
GROUP BY cl.patient_id, diagnosis_code
ORDER BY total_condition_cost DESC;


In [ ]:
/*
Find cost per condition per patient as an efficiency and burden measure.

Interpretation: What is the total and average claim amount per medical condition (diagnosis) per patient over the entire available time period? This will help measure both the total financial burden (total claims) and treatment efficiency (average claim) for each patient's medical conditions.
*/

WITH __patients AS (
  SELECT
    patient_id,
    family_name,
    given_name
  FROM hl7_fhir.harmonized.patients
), __conditions AS (
  SELECT
    patient_id,
    diagnosis_code,
    diagnosis
  FROM hl7_fhir.harmonized.conditions
), __claims AS (
  SELECT
    patient_id,
    claim_amount
  FROM hl7_fhir.harmonized.claims
), patient_condition_claims AS (
  SELECT DISTINCT
    p.patient_id,
    p.given_name,
    p.family_name,
    c.diagnosis_code,
    c.diagnosis,
    cl.claim_amount
  FROM __patients AS p
  LEFT OUTER JOIN __conditions AS c
    ON p.patient_id = c.patient_id
  LEFT OUTER JOIN __claims AS cl
    ON p.patient_id = cl.patient_id
  WHERE
    NOT c.diagnosis_code IS NULL AND NOT cl.claim_amount IS NULL
)
SELECT
  patient_id,
  given_name,
  family_name,
  diagnosis_code,
  diagnosis,
  COUNT(*) AS num_claims,
  SUM(claim_amount) AS total_claims,
  AVG(claim_amount) AS avg_claim_amount
FROM patient_condition_claims
GROUP BY
  patient_id,
  given_name,
  family_name,
  diagnosis_code,
  diagnosis
ORDER BY
  patient_id,
  total_claims DESC NULLS LAST
 -- Generated by Cortex Analyst
;

In [ ]:
/*
What is the yearly trend of total claim amount per claim type?

What is the yearly trend of total claim amount per claim type over the entire available time period? Please show the year, claim type, and total claim amount for each year-claim type combination.
*/

WITH __claims AS (
  SELECT
    claim_type_code,
    claim_created_datetime,
    claim_amount
  FROM hl7_fhir.harmonized.claims
), yearly_claims AS (
  SELECT
    DATE_PART('YEAR', c.claim_created_datetime) AS claim_year,
    c.claim_type_code,
    SUM(c.claim_amount) AS total_claim_amount
  FROM __claims AS c
  GROUP BY
    DATE_PART('YEAR', c.claim_created_datetime),
    c.claim_type_code
)
SELECT
  claim_year,
  claim_type_code,
  total_claim_amount
FROM yearly_claims
ORDER BY
  claim_year DESC NULLS LAST,
  total_claim_amount DESC NULLS LAST
 -- Generated by Cortex Analyst
;

In [ ]:
/*
What is the yearly trend of total claim amount per diagnosis per service provider?

What is the yearly trend of total claim amount broken down by both diagnosis code and service provider name, showing the total claim amount for each combination of year, diagnosis code, and service provider over the entire available time period? This will help analyze how claim amounts vary across different diagnoses and providers over time.

*/

WITH __claims AS (
  SELECT
    encounter_id,
    service_provider_name,
    claim_created_datetime,
    claim_amount
  FROM hl7_fhir.harmonized.claims
), __encounters AS (
  SELECT
    encounter_id
  FROM hl7_fhir.harmonized.encounters
), __conditions AS (
  SELECT
    diagnosis_encounter_id,
    diagnosis_code
  FROM hl7_fhir.harmonized.conditions
), yearly_claims AS (
  SELECT
    DATE_PART('YEAR', c.claim_created_datetime) AS claim_year,
    cond.diagnosis_code,
    c.service_provider_name,
    SUM(c.claim_amount) AS total_claim_amount
  FROM __claims AS c
  LEFT OUTER JOIN __encounters AS e
    ON e.encounter_id = c.encounter_id
  LEFT OUTER JOIN __conditions AS cond
    ON e.encounter_id = cond.diagnosis_encounter_id
  WHERE
    NOT cond.diagnosis_code IS NULL AND NOT c.service_provider_name IS NULL
  GROUP BY
    DATE_PART('YEAR', c.claim_created_datetime),
    cond.diagnosis_code,
    c.service_provider_name
)
SELECT
  claim_year,
  diagnosis_code,
  service_provider_name,
  total_claim_amount
FROM yearly_claims
ORDER BY
  claim_year DESC NULLS LAST,
  total_claim_amount DESC NULLS LAST
 -- Generated by Cortex Analyst
;

📈 __Quality & Outcomes__

Q13. Are patients with chronic conditions (e.g., diabetes) adhering to expected medication patterns? 

Q14. What proportion of diagnosed medical conditions have an associated medication prescribed for treatment? 

In [ ]:
/*
Name: patients with diabetes adhering to expected medication patterns
Question: Are patients with chronic conditions (e.g., diabetes) adhering to expected medication patterns?
Verified by: Lavinia Carabet
Verified at: 2025-06-05 17:08:58
SQL query:

WITH med_patterns AS (SELECT patient_id, COUNT(DISTINCT medication_code) AS num_medications, MIN(medication_request_datetime) AS first_med, MAX(medication_request_datetime) AS last_med, DATEDIFF(YEAR, MIN(medication_request_datetime), MAX(medication_request_datetime)) AS duration_years FROM __medications GROUP BY patient_id), chronic_conditions AS (SELECT patient_id, diagnosis_code, MIN(diagnosis) AS diagnosis FROM __conditions WHERE diagnosis_code = '44054006' GROUP BY patient_id, diagnosis_code) SELECT mp.patient_id, diagnosis_code, diagnosis, num_medications, first_med, last_med, duration_years FROM med_patterns AS mp JOIN chronic_conditions AS cc ON cc.patient_id = mp.patient_id ORDER BY duration_years DESC
*/

WITH med_patterns AS (
    SELECT 
        patient_id,
        COUNT(DISTINCT medication_code) AS num_medications,
        MIN(medication_request_datetime) AS first_med,
        MAX(medication_request_datetime) AS last_med,
        DATEDIFF('year', MIN(medication_request_datetime), MAX(medication_request_datetime)) AS duration_years
    FROM MEDICATIONS
    GROUP BY patient_id
), 
chronic_conditions AS (
    SELECT 
        patient_id, 
        diagnosis_code,
        MIN(diagnosis) AS diagnosis
    FROM CONDITIONS
    WHERE diagnosis_code = '44054006'  -- diabetes
    GROUP BY patient_id, diagnosis_code
)

SELECT 
    mp.patient_id,
    diagnosis_code,
    diagnosis,
    num_medications,
    first_med,
    last_med, 
    duration_years
FROM med_patterns mp
JOIN chronic_conditions cc ON cc.patient_id = mp.patient_id
ORDER BY duration_years DESC;   

In [ ]:
WITH __medications AS (
  SELECT
    patient_id,
    medication_code,
    medication_request_datetime
  FROM hl7_fhir.harmonized.medications
), __conditions AS (
  SELECT
    patient_id,
    diagnosis_code,
    diagnosis
  FROM hl7_fhir.harmonized.conditions
), med_patterns AS (
  SELECT
    patient_id,
    COUNT(DISTINCT medication_code) AS num_medications,
    MIN(medication_request_datetime) AS first_med,
    MAX(medication_request_datetime) AS last_med,
    DATEDIFF(YEAR, MIN(medication_request_datetime), MAX(medication_request_datetime)) AS duration_years
  FROM __medications
  GROUP BY
    patient_id
), chronic_conditions AS (
  SELECT
    patient_id,
    diagnosis_code,
    MIN(diagnosis) AS diagnosis
  FROM __conditions
  WHERE
    diagnosis_code = '44054006' /* diabetes */
  GROUP BY
    patient_id,
    diagnosis_code
)
SELECT
  mp.patient_id,
  diagnosis_code,
  diagnosis,
  num_medications,
  first_med,
  last_med,
  duration_years
FROM med_patterns AS mp
JOIN chronic_conditions AS cc
  ON cc.patient_id = mp.patient_id
ORDER BY
  duration_years DESC
 -- Generated by Cortex Analyst
;

In [ ]:
/*What proportion of diagnosed medical conditions have an associated medication prescribed for treatment? This will be calculated as the ratio of conditions that have a corresponding medication record (matched by patient_id) to the total number of conditions.*/

WITH __conditions AS (
  SELECT
    condition_id,
    patient_id
  FROM hl7_fhir.harmonized.conditions
), __medications AS (
  SELECT
    medication_id,
    patient_id
  FROM hl7_fhir.harmonized.medications
), condition_med_count AS (
  SELECT
    c.condition_id,
    CASE WHEN NOT m.medication_id IS NULL THEN 1 ELSE 0 END AS has_medication
  FROM __conditions AS c
  LEFT OUTER JOIN __medications AS m
    ON c.patient_id = m.patient_id
)
SELECT
  COUNT(CASE WHEN has_medication = 1 THEN condition_id END) / NULLIF(CAST(COUNT(condition_id) AS FLOAT), 0) AS condition_medication_ratio,
  COUNT(CASE WHEN has_medication = 1 THEN condition_id END) AS conditions_with_medication,
  COUNT(condition_id) AS total_conditions
FROM condition_med_count
 -- Generated by Cortex Analyst
;



🧩 __Patient Journey Mapping__

Q15. CTE Chain for Full Patient Journey 

Q16. Can we track a patient's clinical journey through encounters → diagnoses → treatment → claims? 

In [ ]:
-- CTE for Full Patient Journey


WITH patient_journey AS (
    SELECT 
        p.patient_id,
        e.encounter_end_datetime,
        c.diagnosis_code,
        m.medication_code,
        cl.claim_amount,
        cl.claim_created_datetime
    FROM PATIENTS p
    LEFT JOIN ENCOUNTERS e ON p.patient_id = e.patient_id
    LEFT JOIN CONDITIONS c ON e.encounter_id = c.diagnosis_encounter_id
    LEFT JOIN MEDICATIONS m ON e.encounter_id = m.medication_encounter_id
    LEFT JOIN CLAIMS cl ON e.encounter_id = cl.encounter_id
)
SELECT 
    patient_id,
    MIN(encounter_end_datetime) AS first_encounter,
    MAX(claim_created_datetime) AS last_claim,
    COUNT(DISTINCT diagnosis_code) AS num_conditions,
    COUNT(DISTINCT medication_code) AS num_medications,
    SUM(claim_amount) AS total_claim_amount
FROM patient_journey
GROUP BY patient_id
ORDER BY total_claim_amount DESC
LIMIT 50;


In [ ]:
/*
Interpretation: For each patient, show their first encounter date, last claim date, total number of unique conditions diagnosed, total number of unique medications prescribed, and total claim amount over the entire available time period. Return the top 50 patients by total claim amount.
*/

WITH __patients AS (
  SELECT
    patient_id
  FROM hl7_fhir.harmonized.patients
), __encounters AS (
  SELECT
    encounter_id,
    patient_id,
    encounter_end_datetime
  FROM hl7_fhir.harmonized.encounters
), __conditions AS (
  SELECT
    diagnosis_encounter_id,
    diagnosis_code
  FROM hl7_fhir.harmonized.conditions
), __medications AS (
  SELECT
    medication_encounter_id,
    medication_code
  FROM hl7_fhir.harmonized.medications
), __claims AS (
  SELECT
    encounter_id,
    claim_created_datetime,
    claim_amount
  FROM hl7_fhir.harmonized.claims
), patient_journey AS (
  SELECT
    p.patient_id,
    e.encounter_end_datetime,
    c.diagnosis_code,
    m.medication_code,
    cl.claim_amount,
    cl.claim_created_datetime
  FROM __patients AS p
  LEFT OUTER JOIN __encounters AS e
    ON p.patient_id = e.patient_id
  LEFT OUTER JOIN __conditions AS c
    ON e.encounter_id = c.diagnosis_encounter_id
  LEFT OUTER JOIN __medications AS m
    ON e.encounter_id = m.medication_encounter_id
  LEFT OUTER JOIN __claims AS cl
    ON e.encounter_id = cl.encounter_id
)
SELECT
  patient_id,
  MIN(encounter_end_datetime) AS first_encounter,
  MAX(claim_created_datetime) AS last_claim,
  COUNT(DISTINCT diagnosis_code) AS num_conditions,
  COUNT(DISTINCT medication_code) AS num_medications,
  SUM(claim_amount) AS total_claim_amount
FROM patient_journey
GROUP BY
  patient_id
ORDER BY
  total_claim_amount DESC NULLS LAST
LIMIT 50
 -- Generated by Cortex Analyst
;

In [ ]:
-- track a patient's clinical journey through encounters → diagnoses → treatment → claims

SELECT DISTINCT
    p.patient_id,
    p.family_name,
    p.given_name,
    e.encounter_id,
    e.encounter_end_datetime,
    c.diagnosis_code,
    c.diagnosis,
    m.medication_code,
    m.medication,
    cl.claim_id,
    cl.product_service,
    cl.claim_created_datetime,
    cl.claim_amount
FROM PATIENTS p
JOIN ENCOUNTERS e ON p.patient_id = e.patient_id
LEFT JOIN CONDITIONS c ON e.encounter_id = c.diagnosis_encounter_id
LEFT JOIN MEDICATIONS m ON e.encounter_id = m.medication_encounter_id
LEFT JOIN CLAIMS cl ON e.encounter_id = cl.encounter_id
ORDER BY p.patient_id, e.encounter_end_datetime DESC;


In [ ]:
/*
Track a patient's clinical journey through encounters → diagnoses → treatment → claims.

Interpretation: For each patient, show their clinical journey by displaying all their encounters, diagnoses, medications, and claims in chronological order, including the event type (encounter/diagnosis/medication/claim), date, and relevant details (e.g., diagnosis description, medication name, claim amount) over the entire available time period.

*/


WITH __patients AS (
  SELECT
    patient_id,
    family_name,
    given_name
  FROM hl7_fhir.harmonized.patients
), __claims AS (
  SELECT
    patient_id,
    encounter_id,
    product_service,
    claim_created_datetime,
    claim_amount
  FROM hl7_fhir.harmonized.claims
), all_events AS (
  SELECT
    p.patient_id,
    c.encounter_id,
    'Claim' AS event_type,
    c.claim_created_datetime AS event_date,
    c.product_service AS event_detail,
    c.claim_amount AS amount
  FROM __patients AS p
  LEFT OUTER JOIN __claims AS c
    ON p.patient_id = c.patient_id
)
SELECT
  ae.patient_id,
  p.given_name,
  p.family_name,
  ae.event_type,
  ae.event_date,
  ae.event_detail,
  ae.amount
FROM all_events AS ae
JOIN __patients AS p
  ON ae.patient_id = p.patient_id
ORDER BY
  ae.patient_id,
  ae.event_date DESC NULLS LAST
 -- Generated by Cortex Analyst
;